<a href="https://colab.research.google.com/github/kridtapon/Volume-Weighted-MACD-Crossover-System/blob/main/Volume_Weighted_MACD_Crossover_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00


In [1]:
pip install --upgrade yfinance

In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Volume-Weighted Moving Average (VWMA)
def calculate_vwma(df, period=20):
    return (df['Close'] * df['Volume']).rolling(window=period).sum() / df['Volume'].rolling(window=period).sum()

# Function to calculate MACD and Signal Line
def calculate_macd(df, fast=12, slow=26, signal=9):
    df['MACD'] = df['Close'].ewm(span=fast, adjust=False).mean() - df['Close'].ewm(span=slow, adjust=False).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df['MACD'], df['Signal_Line']

# Define the stock symbol and time period
symbol = 'AAPL'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Define the range of parameters for optimization
vwma_periods = np.arange(10, 51)  # Range for VWMA period
macd_fast = np.arange(5, 21)  # Range for MACD fast
macd_slow = np.arange(20, 31)  # Range for MACD slow
macd_signal = np.arange(5, 16)  # Range for MACD signal

# Define the function to calculate the strategy performance
def calculate_strategy(vwma_period, macd_fast, macd_slow, macd_signal):
    # Calculate VWMA and MACD
    df['VWMA'] = calculate_vwma(df, period=vwma_period)
    df['MACD'], df['Signal_Line'] = calculate_macd(df, fast=macd_fast, slow=macd_slow, signal=macd_signal)

    # Define Entry and Exit signals based on VWMA and MACD Crossover
    df['Entry'] = (df['MACD'] > df['Signal_Line']) & (df['Close'] > df['VWMA'])
    df['Exit'] = (df['MACD'] < df['Signal_Line']) & (df['Close'] < df['VWMA'])

    # Filter data for the test period (2020-2025)
    df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    # Return the final portfolio value (equity curve)
    return portfolio.total_return()

# Perform optimization using itertools.product
best_return = -np.inf
best_params = None

for params in itertools.product(vwma_periods, macd_fast, macd_slow, macd_signal):
    vwma_period, fast, slow, signal = params
    strategy_return = calculate_strategy(vwma_period, fast, slow, signal)
    if strategy_return > best_return:
        best_return = strategy_return
        best_params = (vwma_period, fast, slow, signal)

print(f"Best Parameters: VWMA Period = {best_params[0]}, MACD Fast = {best_params[1]}, MACD Slow = {best_params[2]}, MACD Signal = {best_params[3]}")
print(f"Best Return: {best_return}")

# Calculate the portfolio with the best parameters
df['VWMA'] = calculate_vwma(df, period=best_params[0])
df['MACD'], df['Signal_Line'] = calculate_macd(df, fast=best_params[1], slow=best_params[2], signal=best_params[3])
df['Entry'] = (df['MACD'] > df['Signal_Line']) & (df['Close'] > df['VWMA'])
df['Exit'] = (df['MACD'] < df['Signal_Line']) & (df['Close'] < df['VWMA'])

# Filter data for the test period (2020-2025)
df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest with the best parameters
portfolio = vbt.Portfolio.from_signals(
    close=df_filtered['Close'],
    entries=df_filtered['Entry'],
    exits=df_filtered['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed


Best Parameters: VWMA Period = 12, MACD Fast = 9, MACD Slow = 20, MACD Signal = 11
Best Return: 2.684280268605831
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           368428.026861
Total Return [%]                       268.428027
Benchmark Return [%]                   244.002318
Max Gross Exposure [%]                      100.0
Total Fees Paid                      17782.840116
Max Drawdown [%]                        14.433056
Max Drawdown Duration                       200.0
Total Trades                                   37
Total Closed Trades                            37
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            54.054054
Best Trade [%]                          43.845575
Worst Trade [%]                     

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Volume-Weighted Moving Average (VWMA)
def calculate_vwma(df, period=20):
    return (df['Close'] * df['Volume']).rolling(window=period).sum() / df['Volume'].rolling(window=period).sum()

# Function to calculate MACD and Signal Line
def calculate_macd(df, fast=12, slow=26, signal=9):
    df['MACD'] = df['Close'].ewm(span=fast, adjust=False).mean() - df['Close'].ewm(span=slow, adjust=False).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df['MACD'], df['Signal_Line']

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate VWMA and MACD
df['VWMA'] = calculate_vwma(df, period=20)
df['MACD'], df['Signal_Line'] = calculate_macd(df)

# Define Entry and Exit signals based on VWMA and MACD Crossover
df['Entry'] = (df['MACD'] > df['Signal_Line']) & (df['Close'] > df['VWMA'])
df['Exit'] = (df['MACD'] < df['Signal_Line']) & (df['Close'] < df['VWMA'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           105945.951179
Total Return [%]                         5.945951
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       7713.835191
Max Drawdown [%]                        78.080992
Max Drawdown Duration                      1093.0
Total Trades                                   42
Total Closed Trades                            42
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            42.857143
Best Trade [%]                          44.298349
Worst Trade [%]                        -28.947324
Avg Winning Trade [%]                   11.419914
Avg Losing Trade [%]                    -6.769887
